In [2]:
import pandas as pd
from emoji import UNICODE_EMOJI
import emoji
import regex as re
import tweepy
from tqdm import tqdm

In [3]:
# setup API auhorizaion

# assign the values accordingly
consumer_key = "jfpVhd1zOxM6uqR1GjcQzEsns"
consumer_secret = "c4PdCb2fvIbFRHlgQJlT6k0cPlRUtl1E0akP5dSvmKz1NFUHmr"
access_token = "1250984376671670272-3wFUr43oEV9NUiPtv8f7ygVT74Rw3s"
access_token_secret = "2OyDgcbZWYbVwgvFJttSIEroJzmOOTwq9hGMGmVeRs9mg"

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  
# set access to user's access key and access secret 
auth.set_access_token(access_token, access_token_secret)

In [4]:
# set dataframe display settings to show 1000 rows
pd.options.display.max_rows = 1000

In [5]:
# cleans text by removing extraneous characters
def clean(text):
    text = re.sub(r"'s", "", text) # remove possesive 's
    text = re.sub(r"http\S+", " ", text) # remove urls
    text = re.sub(r"RT ", " ", text) # remove rt
    text = re.sub(r"'[s]", "", text) # remove '
    text = re.sub(r"[^a-zA-Z\d\s]", " ", text) # remove special characters
    text = re.sub(r"[0-9]", " ", text) # remove number
    text = re.sub(r'\t', ' ', text) # remove tabs
    text = re.sub(r'\n', ' ', text) # remove line jump
    text = re.sub(r"\s+", " ", text) # remove extra white space
    text = text.strip()
    text = text.lower()
    return text

In [6]:
# count emojis in tweet text
def split_count(text):
    total_emoji = []
    data = re.findall(r'\X',text)
    flag = False
    for word in data:
        if word in emoji.UNICODE_EMOJI['en']:  
            total_emoji += [word] # total_emoji is a list of all emojis

    # remove from the given text the emojis
    for current in total_emoji:
        text = text.replace(current, '') 

    # return Counter(text.split() + total_emoji)
    return len(total_emoji)

In [7]:
# counts urls in tweet text
def countUrls(text):
    return len(re.findall(r"http\S+", text))

In [8]:
# counts hashtags in tweet text
def countHashtags(text):
    return len(re.findall(r"#[a-zA-Z]+", text))

In [9]:
# get list or count of followers for a specific user
def getFollowers(user, list=False):
    if list:
        return user.followers
    else:
        return user.followers_count

In [10]:
# get list or count of statuses for a specific user
def getStatuses(user, list=False):
    if list:
        return user.statuses
    else:
        return user.statuses_count

In [11]:
# get list or count of friends (people followed) for a specific user
def getFriends(user, list=False):
    if list:
        return user.friends
    else:
        return user.friends_count

In [12]:
# get list or count of friends (people followed) for a specific user
def getVerified(user):
    return user.verified

In [ ]:
# load verifed humans twitter accounts dataset
verifiedHumans = pd.read_csv('verified-2019.tsv', header=None, sep='\t')
verifiedHumans

In [ ]:
# get the ids from the first column
userIds = verifiedHumans[0]

In [ ]:
# feature lists
ids = []
human = []
rawText = []
text = []
emojis = []
followers = []
following = []
hashtags = []
urls = []
statuses = []
verified = []

In [23]:
# calling the twitter api 
api = tweepy.API(auth)

In [ ]:
# construct the training set by calling predefined functions
for id in tqdm(userIds):
    # Put in a try-except block in case a forbidden user id is in there
    try:
        print(id)
        user = api.get_user(user_id=id)

        recentFiveTweets = api.user_timeline(user_id=id, 
                                # 200 is the maximum allowed count
                                count=11,
                                include_rts = True,
                                # Necessary to keep full_text 
                                # otherwise only the first 140 words are extracted
                                tweet_mode = 'extended'
                                )
        fullText = ""
        for info in recentFiveTweets:
            fullText = fullText + info.full_text + " "
        cleanedText = clean(fullText)
        
        ids.append(id)
        emojis.append(split_count(fullText))
        followers.append(getFollowers(user))
        following.append(getFriends(user))
        hashtags.append(countHashtags(fullText))
        urls.append(countUrls(fullText))
        statuses.append(getStatuses(user))
        verified.append(getVerified(user))
        text.append(cleanedText)
        rawText.append(fullText)
        human.append(1)
    except:
        pass
    

In [ ]:
# assemble data dictionary
dataDict = {
    "id": ids,
    "rawText": rawText,
    "cleanText": text,
    "emojiCount": emojis,
    "followers": followers,
    "following": following,
    "hashtagCount": hashtags,
    "urlCount": urls,
    "tweetCount": statuses,
    "verified": verified,
    "human": human
}

In [ ]:
# turn dict into dataframe for export
trainData = pd.DataFrame.from_dict(dataDict)

In [ ]:
# now we will repeat the same steps for the bot dataset

In [13]:
trainData.to_csv(r'/Users/lincolnrychecky/desktop/trainingData.csv', index = False)

NameError: name 'trainData' is not defined

In [43]:
# load verifed bots twitter accounts dataset
verifiedBots = pd.read_csv('bots-2017.tsv', header=None, sep='\t').head(1130)
verifiedBots

,0,1
0,461277906,bot
1,808280162,bot
2,1575510320,bot
3,2449235599,bot
4,1113769100,bot
...,...,...
1125,440784095,bot
1126,297174372,bot
1127,112431151,bot
1128,14614833,bot


In [44]:
# load verifed bots twitter accounts dataset
verifiedBotsTwo = pd.read_csv('bots-2019.tsv', header=None, sep='\t')
verifiedBotsTwo

,0,1
0,1120973992754581509,bot
1,999622983424598016,bot
2,1118951206448181248,bot
3,1114680860983214081,bot
4,1028578680187052032,bot
5,4181648555,bot
6,1101849622073491456,bot
7,41681878,bot
8,1099737526686937088,bot
9,1109778441379950592,bot


In [46]:
frames = [verifiedBots, verifiedBotsTwo]

In [48]:
result = pd.concat(frames)

In [49]:
result

,0,1
0,461277906,bot
1,808280162,bot
2,1575510320,bot
3,2449235599,bot
4,1113769100,bot
...,...,...
699,4187564112,bot
700,831720146091716608,bot
701,4002930083,bot
702,708794198099693568,bot


In [50]:
# get the ids from the first column
userIds = result[0]

In [52]:
# feature lists
ids = []
human = []
rawText = []
text = []
emojis = []
followers = []
following = []
hashtags = []
urls = []
statuses = []
verified = []

In [53]:
# construct the training set by calling predefined functions
for id in tqdm(userIds):
    # Put in a try-except block in case a forbidden user id is in there
    try:
        print(id)
        user = api.get_user(user_id=id)

        recentFiveTweets = api.user_timeline(user_id=id, 
                                # 200 is the maximum allowed count
                                count=11,
                                include_rts = True,
                                # Necessary to keep full_text 
                                # otherwise only the first 140 words are extracted
                                tweet_mode = 'extended'
                                )
        fullText = ""
        for info in recentFiveTweets:
            fullText = fullText + info.full_text + " "
        cleanedText = clean(fullText)
        
        ids.append(id)
        emojis.append(split_count(fullText))
        followers.append(getFollowers(user))
        following.append(getFriends(user))
        hashtags.append(countHashtags(fullText))
        urls.append(countUrls(fullText))
        statuses.append(getStatuses(user))
        verified.append(getVerified(user))
        text.append(cleanedText)
        rawText.append(fullText)
        human.append(0)
    except:
        pass
    

  0%|                                                  | 0/1834 [00:00<?, ?it/s]

461277906


  0%|                                          | 1/1834 [00:00<14:06,  2.17it/s]

808280162


  0%|                                          | 2/1834 [00:00<13:36,  2.24it/s]

...

100%|██████████████████████████████████████▉| 1831/1834 [12:36<00:01,  2.23it/s]

4002930083


100%|██████████████████████████████████████▉| 1833/1834 [12:36<00:00,  2.69it/s]

708794198099693568
757804429357613056


100%|███████████████████████████████████████| 1834/1834 [12:37<00:00,  2.42it/s]


In [54]:
# assemble data dictionary
dataDict = {
    "id": ids,
    "rawText": rawText,
    "cleanText": text,
    "emojiCount": emojis,
    "followers": followers,
    "following": following,
    "hashtagCount": hashtags,
    "urlCount": urls,
    "tweetCount": statuses,
    "verified": verified,
    "human": human
}

In [55]:
# turn dict into dataframe for export
trainDataBot = pd.DataFrame.from_dict(dataDict)

In [56]:
trainDataBot.to_csv(r'/Users/lincolnrychecky/desktop/trainingDataBots.csv', index = False)

In [ ]:
def buildPredictionSet(userIds):
    # construct the training set by calling predefined functions
    for id in tqdm(userIds):
        # Put in a try-except block in case a forbidden user id is in there
        try:
            print(id)
            user = api.get_user(user_id=id)

            recentFiveTweets = api.user_timeline(user_id=id, 
                                    # 200 is the maximum allowed count
                                    count=11,
                                    include_rts = True,
                                    # Necessary to keep full_text 
                                    # otherwise only the first 140 words are extracted
                                    tweet_mode = 'extended'
                                    )
            fullText = ""
            for info in recentFiveTweets:
                fullText = fullText + info.full_text + " "
            cleanedText = clean(fullText)

            ids.append(id)
            emojis.append(split_count(fullText))
            followers.append(getFollowers(user))
            following.append(getFriends(user))
            hashtags.append(countHashtags(fullText))
            urls.append(countUrls(fullText))
            statuses.append(getStatuses(user))
            verified.append(getVerified(user))
            text.append(cleanedText)
            rawText.append(fullText)
            human.append(0)
        except:
            pass
        
    # turn dict into dataframe for export
    trainDataBot = pd.DataFrame.from_dict(dataDict)
    trainDataBot.to_csv(r'/Users/lincolnrychecky/desktop/trainingDataBots.csv', index = False)